# LGMC (static) - Electrocatalytic CO oxidation on Cu

Simple example of [electrocatalytic CO oxidation on Cu](./https://pubs.aip.org/aip/jcp/article/155/16/164701/199778/Effects-of-surface-diffusion-in-electrocatalytic). 

In [18]:
import sqlite3

In [19]:
## TABLES FOR LATTICE REACTION NETWORK DATABASE ##

create_metadata_table = """
    CREATE TABLE metadata (
            number_of_species   INTEGER NOT NULL,
            number_of_reactions INTEGER NOT NULL
    );
"""

insert_metadata = """
    INSERT INTO metadata VALUES (?, ?)
"""

# it is important that reaction_id is the primary key
# otherwise the network loader will be extremely slow.

create_reactions_table = """
    CREATE TABLE reactions (
            reaction_id                     INTEGER NOT NULL PRIMARY KEY,
            number_of_reactants             INTEGER NOT NULL,
            number_of_products              INTEGER NOT NULL,
            reactant_1                      INTEGER NOT NULL,
            reactant_2                      INTEGER NOT NULL,
            product_1                       INTEGER NOT NULL,
            product_2                       INTEGER NOT NULL,
            phase_reactant_1                CHAR(1) NOT NULL,
            phase_reactant_2                CHAR(1) NOT NULL,
            phase_product_1                 CHAR(1) NOT NULL,
            phase_product_2                 CHAR(1) NOT NULL,
            dG                              REAL NOT NULL,
            prefactor                       REAL NOT NULL,
            rate                            REAL NOT NULL,
            electron_tunneling_coefficient  REAL NOT NULL,
            reorganization_energy           REAL NOT NULL,
            charge_transfer_coefficient     REAL NOT NULL,
            type                            CHAR(1) NOT NULL
    );
"""

insert_reaction = """
    INSERT INTO reactions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [20]:
## TABLES FOR STATE DATABASE ##

create_initial_state_table = """
    CREATE TABLE initial_state (
            species_id             INTEGER NOT NULL PRIMARY KEY,
            count                  INTEGER NOT NULL
    );
"""

create_trajectories_table = """
    CREATE TABLE trajectories (
            seed                INTEGER NOT NULL,
            step                INTEGER NOT NULL,
            time                REAL NOT NULL,
            reaction_id         INTEGER NOT NULL,
            site_1_mapping      INTEGER NOT NULL,
            site_2_mapping      INTEGER NOT NULL
    );
"""


create_factors_table = """
    CREATE TABLE factors (
            factor_zero      REAL NOT NULL,
            factor_two       REAL NOT NULL,
            factor_duplicate REAL NOT NULL
    );
"""


create_interrupt_state = """
    CREATE TABLE interrupt_state (
            seed                    INTEGER NOT NULL,
            species_id              INTEGER NOT NULL,
            quantity                INTEGER NOT NULL,
            site_mapping            INTEGER NOT NULL,
            edge                    INTEGER NOT NULL
            
    );
"""

create_interrupt_cutoff = """
    CREATE TABLE interrupt_cutoff (
            seed                    INTEGER NOT NULL,
            step                    INTEGER NOT NULL,
            time                    INTEGER NOT NULL,
            maxk                    INTEGER NOT NULL
            
    );
"""

In [21]:
## FUNCTION TO CREATE INITIAL STATE DATABASE ##

def insert_initial_state(
        initial_state,
        mol_entries,
        initial_state_db,
        factor_zero = 1.0,
        factor_two = 1.0,
        factor_duplicate = 0.5
):
    """
    create initial state database with initial_state, trajectories,
    factors, interrupt_state, and interrupt_cutoff tables
    """

    rn_con = sqlite3.connect(initial_state_db)
    rn_cur = rn_con.cursor()
    rn_cur.execute(create_initial_state_table)
    rn_cur.execute(create_trajectories_table)
    rn_cur.execute(create_factors_table)
    rn_cur.execute(create_interrupt_state)
    rn_cur.execute(create_interrupt_cutoff)
    rn_con.commit()

    rn_cur.execute(
        "INSERT INTO factors VALUES (?,?,?)",
        (factor_zero, factor_two, factor_duplicate))

    num_species = len(mol_entries)


    for i in range(num_species):
        rn_cur.execute(
            "INSERT INTO initial_state VALUES (?,?)",
            (i, initial_state.get(i,0)))

    rn_con.commit()

In [22]:
## CREATE INITIAL STATE DATABASE ##

insert_initial_state({0:0,
                      1:2500,
                      2:0,
                      3:0,
                      4:15000},
                     [0, 0, 0, 0, 0],
                     "initial_state.sqlite")

In [17]:
## CREATE LATTICE REACTION NETWORK DATABASE ##

reactions = [
    # CO(elyte) + * -> CO*
    [0, 2, 1, 0, 1, 1, -1, "L", "S", "L", "N", -100.0, 1.0, 10e5, 0.0, 0.0, 0.0, "A"],
    # * + H2O(elyte) -> H2O*
    [1, 2, 1, 0, 4, 4, -1, "L", "S", "L", "N", -100.0, 1.0, 10e2, 0.0, 0.0, 0.0, "A"],
    # H2O* -> * + H2O(elyte)
    [2, 1, 2, 4, -1, 0, 4, "L", "N", "L", "S", -100.0, 1.0, 10e2, 0.0, 0.0, 0.0, "D"],
    # H2O* -> OH* + e-
    [3, 1, 1, 4, -1, 3, -1, "L", "N", "L", "N", 0.0, 0.02, 0.0, 1.2, 0.0, 0.5, "O"],
    # OH* + e- -> H2O*
    [4, 1, 1, 3, -1, 4, -1, "L", "N", "L", "N", 0.0, 10e4, 0.0, 1.2, 0.0, 0.5, "R"],
    # CO* + OH* -> CO2* + e-
    [5, 2, 2, 1, 3, 2, 0, "L", "L", "L", "L", 0.0, 0.8432, 0.0, 1.2, 0.0, 0.5, "O"],
    # CO2* -> * + CO2(elyte)
    [6, 1, 2, 2, -1, 0, 2, "L", "N", "L", "S", -100.0, 1.0, 10e4, 0.0, 0.0, 0.0, "D"],
    # CO* diffusion
    [7, 2, 2, 0, 1, 0, 1, "L", "L", "L", "L", 0.0, 1.0, 1, 0.0, 0.0, 0.0, "F"],
    # OH* diffusion
    [8, 2, 2, 0, 4, 0, 4, "L", "L", "L", "L", 0.0, 1.0, 1, 0.0, 0.0, 0.0, "F"],
]

rn_con = sqlite3.connect("rn.sqlite")
rn_cur = rn_con.cursor()
rn_cur.execute(create_metadata_table)
rn_cur.execute(create_reactions_table)
rn_con.commit()

rn_cur.execute(insert_metadata, (5, 9))
for reaction in reactions:
    rn_cur.execute(insert_reaction, tuple(reaction))
rn_con.commit()